# Libraries

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None
import numpy as np
import ast
import re
from sklearn.metrics.pairwise import cosine_similarity
import copy

In [4]:
def ExtractUsefulAttributes(column, saraksts, margin=50):
    """
    Definējam funkciju, kas izvilks derīgos atribūtus no kolonnām ar formātu string, kas līdzīgs sarakstam.
    column: kolonna, no kuras izvilksim bieži pārstāvēto elementu sarakstu
    saraksts: ja mums jau ir saraksts ar elementiem, ko gribam
    margin: robeža, zem kuras mēs elementu neuzskatām par gana bieži pārstāvētu
    """
    if saraksts:
        uniqueList = saraksts
    else:
        uniqueList = []
        for i in column.str.replace(' ','').str.strip('[]').str.split(','):
            for z in i:
                if z not in uniqueList and z!='':
                    uniqueList.append(z)
    cnts = []
    for item in uniqueList:
        skaits= 0
        for i in column:
            if item in i:
                skaits = skaits + 1
        cnts.append(skaits)
    tempDF = pd.DataFrame(uniqueList, cnts)
    result = tempDF.loc[tempDF.index > margin][0].to_list()
    print('attribute cnt extracted from column is {}'.format(len(result)))
    return result

#calculate list-like parameter values for content
def ExtractUsefulAttributes_wiki(column, saraksts, margin=30):
    """
    Definējam funkciju, kas izvilks derīgos atribūtus no kolonnām ar formātu string, kas līdzīgs sarakstam.
    column: kolonna, no kuras izvilksim bieži pārstāvēto elementu sarakstu
    saraksts: ja mums jau ir saraksts ar elementiem, ko gribam
    margin: robeža, zem kuras mēs elementu neuzskatām par gana bieži pārstāvētu
    """
    if saraksts:
        uniqueList = saraksts
    else:
        uniqueList = []
        for i in column:
            for el in i:
                if el not in uniqueList and el not in ['','_Jr.']:
                    uniqueList.append(el)
    cnts = []
    for item in uniqueList:
        skaits= 0
        for val in column:
            if item in val:
                skaits = skaits + 1
        cnts.append(skaits)
    tempDF = pd.DataFrame(uniqueList, cnts)
    result = tempDF.loc[tempDF.index > margin][0].to_list()
    print('attribute cnt extracted from column is {}'.format(len(result)))
    return result

# EPG nosaukumu apstrāde

In [5]:
#filmu ielasīšana, skaits, datu piemērs
df = pd.read_csv(r"\Train\epg_movies_content.csv")
print(df.shape)
df.drop(columns=['title_full_en_pre','title_full_lv_pre','title_full_ru_pre'],inplace=True)
df.drop_duplicates(inplace=True)
df.rename(columns={'title_full_original_pre':'movieId'},inplace=True)
print(df.shape)
df.head(1)

(26993, 17)
(25045, 14)


,movieId,hour,is_weekend,channel_name,censure,genre_en,duration,title_main,genre_name,languages,location.value,released.value,cast.value,directors.value
0,turbo,6-9,Jā,TV3 Mini,NaN,"[Adventure,Comedy,Family,Sport,Animation]",1.83,TV3 Mini,Bērniem,"[lv,Orig]",Denmark,2012.0,Elliott Crosset Hove,Andreas Thaulow


In [6]:
after_cleaning_grp=copy.deepcopy(df)
after_cleaning_grp.shape

(25045, 14)

In [7]:
#dublikātu izmešana viena kanāla ietvaros - pēc ilgākā pārraides laika
after_cleaning_grp.sort_values(by=['movieId','title_main','duration'],
                  inplace=True,ascending=False)
after_cleaning_grp.drop_duplicates(subset=['movieId','title_main'],inplace=True,keep='first')

# Pazīmju apstrāde un sagatavošana cosine similarity algoritmam

In [8]:
after_cleaning_grp['languages'] = after_cleaning_grp['languages'].fillna("[]")
after_cleaning_grp['is_weekend'].replace({'Jā':1,'Nē':0},inplace=True)
after_cleaning_grp['censure'] = after_cleaning_grp['censure'].fillna(0).replace({18:16})
cut_labels_4 = [ 1950, 1980, 1990, 2000, 2010, 2015, 2020, 2030]
cut_bins = [1900, 1950, 1980, 1990, 2000, 2010, 2015, 2020, 2030]
after_cleaning_grp['YEAR']=pd.cut(after_cleaning_grp['released.value'], bins=cut_bins, labels=cut_labels_4)

In [9]:
after_cleaning_grp['cast.value'] = after_cleaning_grp['cast.value'].fillna('').str.replace(' ','_').str.split(',')
after_cleaning_grp['directors.value'] = after_cleaning_grp['directors.value'].fillna('').str.replace(' ','_').str.split(',')
after_cleaning_grp['location.value'] = after_cleaning_grp['location.value'].fillna('').str.replace(' ','_').str.split(',')
after_cleaning_grp['released.value'] = after_cleaning_grp['released.value'].fillna(0).astype(int)

In [10]:
dfFinal = after_cleaning_grp[['movieId']]

#calculate list-like parameter values for content
list_parameters=['genre_en','languages']
for item in list_parameters:
    print(item)
    setting = ExtractUsefulAttributes(after_cleaning_grp[item], False)
    for keyword in setting:
        keywList = []
        for i in after_cleaning_grp[item]:
            if keyword in i:
                skaits = 2
            else:
                skaits = 0
            keywList.append(skaits)
        dfFinal['_'.join([item,keyword])] = keywList
        
#calculate list-like parameter values for content_wiki
list_parameters=['location.value','cast.value']
for item in list_parameters:
    print(item)
    setting = ExtractUsefulAttributes_wiki(after_cleaning_grp[item], False, margin=50)
    for keyword in setting:
        keywList = []
        for i in after_cleaning_grp[item]:
            if keyword in i:
                skaits = 1
            else:
                skaits = 0
            keywList.append(skaits)
        dfFinal['_'.join([item,keyword])] = keywList
#calculate list-like parameter values for content_wiki
list_parameters=['directors.value']
for item in list_parameters:
    print(item)
    setting = ExtractUsefulAttributes_wiki(after_cleaning_grp[item], False, margin=15)
    for keyword in setting:
        keywList = []
        for i in after_cleaning_grp[item]:
            if keyword in i:
                skaits = 1
            else:
                skaits = 0
            keywList.append(skaits)
        dfFinal['_'.join([item,keyword])] = keywList

#calculate parameter values for content
list_parameters=['channel_name','genre_name','hour','censure','is_weekend','YEAR']
for item in list_parameters:
    print(item)
    setting = after_cleaning_grp[item].astype(str).unique()
    for keyword in setting:
        keywList = []
        for i in after_cleaning_grp[item]:
            if keyword==i:
                skaits = 1
            else:
                skaits = 0
            keywList.append(skaits)
        dfFinal['_'.join([item,keyword])] = keywList
    
cols_to_drop=['censure_0.0','YEAR_nan']
for column in dfFinal.columns:
    if column in cols_to_drop:
        dfFinal.drop(columns = [column],inplace=True)
    

dfFinal2 = dfFinal.drop(columns = ['movieId']).astype(float)

genre_en
attribute cnt extracted from column is 20
languages
attribute cnt extracted from column is 4
location.value
attribute cnt extracted from column is 9
cast.value
attribute cnt extracted from column is 18
directors.value
attribute cnt extracted from column is 23
channel_name
genre_name
hour
censure
is_weekend
YEAR


In [12]:
dfFinal.columns

Index(['movieId', 'genre_en_Drama', 'genre_en_Romance', 'genre_en_Comedy',
       'genre_en_Family', 'genre_en_War', 'genre_en_Sport', 'genre_en_Sci-Fi',
       'genre_en_Adventure', 'genre_en_Fantasy',
       ...
       'is_weekend_0', 'is_weekend_1', 'YEAR_2020.0', 'YEAR_2015.0',
       'YEAR_2000.0', 'YEAR_2010.0', 'YEAR_1980.0', 'YEAR_1990.0',
       'YEAR_2030.0', 'YEAR_1950.0'],
      dtype='object', length=179)

In [13]:
dfFinal.shape

(5698, 179)

In [14]:
dfFinal2.head()

,genre_en_Drama,genre_en_Romance,genre_en_Comedy,genre_en_Family,genre_en_War,genre_en_Sport,genre_en_Sci-Fi,genre_en_Adventure,genre_en_Fantasy,genre_en_Crime,genre_en_Mystery,genre_en_Thriller,genre_en_Animation,genre_en_Action,genre_en_Horror,genre_en_Musical,genre_en_Historical,genre_en_Documentary,genre_en_Music,genre_en_Biography,languages_ru,languages_Orig,languages_lv,languages_en,location.value_Russia,location.value_United_States_of_America,location.value_France,location.value_Soviet_Union,location.value_Canada,location.value_Australia,location.value_Italy,location.value_United_Kingdom,location.value_Germany,cast.value_Owen_Wilson,cast.value_Matt_Damon,cast.value_Samuel_L._Jackson,cast.value_Sylvester_Stallone,cast.value_Octavia_Spencer,cast.value_Woody_Harrelson,cast.value_Morgan_Freeman,cast.value_Liam_Neeson,cast.value_Adam_Sandler,cast.value_Michael_Papajohn,cast.value_Johnny_Depp,cast.value_James_Franco,cast.value_Bruce_Willis,cast.value_Elizabeth_Banks,cast.value_Dwayne_Johnson,cast.value_Mark_Wahlberg,cast.value_Stan_Lee,cast.value_Stanley_Tucci,directors.value_Dmitry_Kiselyov,directors.value_Renny_Harlin,directors.value_Adam_McKay,directors.value_Paul_Feig,directors.value_Will_Gluck,directors.value_Robert_Rodriguez,directors.value_Peter_Segal,directors.value_Dennis_Dugan,directors.value_Marc_Forster,directors.value_Woody_Allen,directors.value_Roland_Emmerich,directors.value_Ridley_Scott,directors.value_Steven_Spielberg,directors.value_Peter_Berg,directors.value_Michael_Bay,directors.value_Francis_Lawrence,directors.value_Ivan_Reitman,directors.value_Andy_Tennant,directors.value_Sam_Raimi,directors.value_Ron_Howard,directors.value_Jay_Roach,directors.value_Chris_Columbus,directors.value_Robert_Schwentke,channel_name_RTVi,channel_name_3+,channel_name_8TV,channel_name_Родное кино,channel_name_Мир,channel_name_Кинокомедия,channel_name_Киномикс,channel_name_TV3 Mini,channel_name_Русский Иллюзион,channel_name_Дом Кино Премиум,channel_name_Дом Кино,channel_name_Мосфильм - золотая коллекция,channel_name_Mosfilm,channel_name_TV3,channel_name_TV3 Film HD,channel_name_Первый Балтийский Канал Латвия,channel_name_TV6,channel_name_Моя планета,channel_name_ТВ Центр international,channel_name_Duo 7,channel_name_Первый Балтийский Kанал,channel_name_Наука,channel_name_Первый Балтийский Kанал Эстония,channel_name_Мультиландия,channel_name_Ретро,channel_name_Иллюзион +,channel_name_В гостях у сказки,channel_name_Sony Channel,channel_name_Filmzone+,channel_name_Filmzone,channel_name_Sony Turbo,channel_name_LTV1,channel_name_TV1000,channel_name_360TV,channel_name_VIP Comedy,channel_name_TV1000 World kino,channel_name_TV1000 Action,channel_name_TV3 Life,channel_name_TV1000 Русское кино,channel_name_Viasat History,channel_name_SAT1,channel_name_Nat Geo Wild,channel_name_FOX Life,channel_name_Animal Planet,channel_name_Investigation Discovery,channel_name_LTV7,channel_name_STV Pirmā!,channel_name_THT,channel_name_Duo 3,channel_name_Filmbox,channel_name_PRO 7,channel_name_Nickelodeon HD,channel_name_Nickelodeon,channel_name_Animal Planet HD,channel_name_Epic Drama HD,channel_name_National Geographic HD,channel_name_National Geographic,channel_name_Discovery Science,channel_name_Discovery Channel HD,channel_name_Discovery Channel,channel_name_FOX,channel_name_РТР-Планета,channel_name_TV1000 Russkoje Kino,channel_name_BBC Earth HD,channel_name_Arte HD,channel_name_Duo 6,channel_name_НТВ-Мир,channel_name_Kidzone,channel_name_Disney Channel,channel_name_AMC,channel_name_TV24,channel_name_РЕН Балтия,channel_name_RTL,channel_name_Setanta Sports HD,channel_name_Setanta Sports,channel_name_Nick Jr,channel_name_Discovery HD Showcase,genre_name_nan,genre_name_Filmas un seriāli,genre_name_Izklaide un dzīvesstils,genre_name_Bērniem,genre_name_Populārzinātniskie,genre_name_Starptautiskie,genre_name_Krievijas,genre_name_Sports,hour_12-18,hour_18-23,hour_23-01,hour_1-6,hour_6-9,hour_9-12,censure_12.0,censure_7.0,censure_16.0,is_weekend_0,is_weekend_1

In [15]:
dfFinal2.shape

(5698, 178)

In [21]:
#tiek atlasīti vienumi, kas satur vismaz 5 pazīmes
nonzeros = (dfFinal2 == 0).astype(int).sum(axis=1)
dfFinal2 ["all_zeros"]= nonzeros.to_list()
dfFinal2 = dfFinal2.loc[dfFinal2["all_zeros"] < dfFinal2.shape[1]-5].drop(columns="all_zeros")
dfFinal3 = dfFinal[dfFinal.index.isin(dfFinal2.index)]

In [25]:
#aprēķināta kosinusa līdzības matrica
cosine_sim = cosine_similarity(dfFinal2)

In [26]:
cosine_sim

array([[1.        , 0.90909091, 0.49236596, ..., 0.53346507, 0.78258558,
        0.35564338],
       [0.90909091, 1.        , 0.55391171, ..., 0.53346507, 0.88040878,
        0.35564338],
       [0.49236596, 0.55391171, 1.        , ..., 0.24077171, 0.59603956,
        0.24077171],
       ...,
       [0.53346507, 0.53346507, 0.24077171, ..., 1.        , 0.62187434,
        0.39130435],
       [0.78258558, 0.88040878, 0.59603956, ..., 0.62187434, 1.        ,
        0.3826919 ],
       [0.35564338, 0.35564338, 0.24077171, ..., 0.39130435, 0.3826919 ,
        1.        ]])

In [27]:
#datu transformācija uz pandas DF
v2 = pd.DataFrame(cosine_sim, columns=dfFinal3.movieId, index=dfFinal3.movieId)

In [28]:
#top 10 vienumu atlase
nlargest = 10
order = np.argsort(-v2.values, axis=1)[:, :nlargest]
v2.columns[order]
result = pd.DataFrame(v2.columns[order], 
                      columns=['top{}'.format(i) for i in range(1, nlargest+1)],
                      index=v2.index)

C:\Aplikacijas\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  after removing the cwd from sys.path.
C:\Aplikacijas\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  """


In [46]:
#piemērs līdzīgajām filmām
result[result.index=='pirates of the caribbean the curse of the black pearl'].head(1)

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
movieId,,,,,,,,,,
pirates of the caribbean the curse of the black pearl,pirates of the caribbean at world’s end,pirates of the caribbean dead man's chest,pirates of the caribbean dead men tell no tales,pirates of the caribbean on stranger tides,pirates of the caribbean the curse of the black pearl,the forbidden kingdom,prince of persia the sands of time,seventh son,wrath of the titans,seventh son


In [45]:
#piemērs līdzīgajām filmām filmas raksturojums
dfFinal[dfFinal.movieId=='pirates of the caribbean the curse of the black pearl'].head(1)

,movieId,genre_en_Drama,genre_en_Romance,genre_en_Comedy,genre_en_Family,genre_en_War,genre_en_Sport,genre_en_Sci-Fi,genre_en_Adventure,genre_en_Fantasy,genre_en_Crime,genre_en_Mystery,genre_en_Thriller,genre_en_Animation,genre_en_Action,genre_en_Horror,genre_en_Musical,genre_en_Historical,genre_en_Documentary,genre_en_Music,genre_en_Biography,languages_ru,languages_Orig,languages_lv,languages_en,location.value_Russia,location.value_United_States_of_America,location.value_France,location.value_Soviet_Union,location.value_Canada,location.value_Australia,location.value_Italy,location.value_United_Kingdom,location.value_Germany,cast.value_Owen_Wilson,cast.value_Matt_Damon,cast.value_Samuel_L._Jackson,cast.value_Sylvester_Stallone,cast.value_Octavia_Spencer,cast.value_Woody_Harrelson,cast.value_Morgan_Freeman,cast.value_Liam_Neeson,cast.value_Adam_Sandler,cast.value_Michael_Papajohn,cast.value_Johnny_Depp,cast.value_James_Franco,cast.value_Bruce_Willis,cast.value_Elizabeth_Banks,cast.value_Dwayne_Johnson,cast.value_Mark_Wahlberg,cast.value_Stan_Lee,cast.value_Stanley_Tucci,directors.value_Dmitry_Kiselyov,directors.value_Renny_Harlin,directors.value_Adam_McKay,directors.value_Paul_Feig,directors.value_Will_Gluck,directors.value_Robert_Rodriguez,directors.value_Peter_Segal,directors.value_Dennis_Dugan,directors.value_Marc_Forster,directors.value_Woody_Allen,directors.value_Roland_Emmerich,directors.value_Ridley_Scott,directors.value_Steven_Spielberg,directors.value_Peter_Berg,directors.value_Michael_Bay,directors.value_Francis_Lawrence,directors.value_Ivan_Reitman,directors.value_Andy_Tennant,directors.value_Sam_Raimi,directors.value_Ron_Howard,directors.value_Jay_Roach,directors.value_Chris_Columbus,directors.value_Robert_Schwentke,channel_name_RTVi,channel_name_3+,channel_name_8TV,channel_name_Родное кино,channel_name_Мир,channel_name_Кинокомедия,channel_name_Киномикс,channel_name_TV3 Mini,channel_name_Русский Иллюзион,channel_name_Дом Кино Премиум,channel_name_Дом Кино,channel_name_Мосфильм - золотая коллекция,channel_name_Mosfilm,channel_name_TV3,channel_name_TV3 Film HD,channel_name_Первый Балтийский Канал Латвия,channel_name_TV6,channel_name_Моя планета,channel_name_ТВ Центр international,channel_name_Duo 7,channel_name_Первый Балтийский Kанал,channel_name_Наука,channel_name_Первый Балтийский Kанал Эстония,channel_name_Мультиландия,channel_name_Ретро,channel_name_Иллюзион +,channel_name_В гостях у сказки,channel_name_Sony Channel,channel_name_Filmzone+,channel_name_Filmzone,channel_name_Sony Turbo,channel_name_LTV1,channel_name_TV1000,channel_name_360TV,channel_name_VIP Comedy,channel_name_TV1000 World kino,channel_name_TV1000 Action,channel_name_TV3 Life,channel_name_TV1000 Русское кино,channel_name_Viasat History,channel_name_SAT1,channel_name_Nat Geo Wild,channel_name_FOX Life,channel_name_Animal Planet,channel_name_Investigation Discovery,channel_name_LTV7,channel_name_STV Pirmā!,channel_name_THT,channel_name_Duo 3,channel_name_Filmbox,channel_name_PRO 7,channel_name_Nickelodeon HD,channel_name_Nickelodeon,channel_name_Animal Planet HD,channel_name_Epic Drama HD,channel_name_National Geographic HD,channel_name_National Geographic,channel_name_Discovery Science,channel_name_Discovery Channel HD,channel_name_Discovery Channel,channel_name_FOX,channel_name_РТР-Планета,channel_name_TV1000 Russkoje Kino,channel_name_BBC Earth HD,channel_name_Arte HD,channel_name_Duo 6,channel_name_НТВ-Мир,channel_name_Kidzone,channel_name_Disney Channel,channel_name_AMC,channel_name_TV24,channel_name_РЕН Балтия,channel_name_RTL,channel_name_Setanta Sports HD,channel_name_Setanta Sports,channel_name_Nick Jr,channel_name_Discovery HD Showcase,genre_name_nan,genre_name_Filmas un seriāli,genre_name_Izklaide un dzīvesstils,genre_name_Bērniem,genre_name_Populārzinātniskie,genre_name_Starptautiskie,genre_name_Krievijas,genre_name_Sports,hour_12-18,hour_18-23,hour_23-01,hour_1-6,hour_6-9,hour_9-12,censure_12.0,censure_7.0,censure_16.0,is_weekend_0,is_w